In [ ]:
import pandas as pd
import multiprocessing as mp
import numpy as np
# import datetime
logs = pd.read_csv("./data_nov/logs_nov.csv")
proc = pd.read_csv("./data_nov/processes_nov.csv")
orders = pd.read_csv("./data_nov/orders_nov.csv")
logs.time_out = pd.to_datetime(logs.time_out, infer_datetime_format=True)
orders.set_index('order_number', inplace=True)

In [ ]:
vip_orders = orders[(~orders.product_length.isnull()) & (orders.order_status == 55) & orders.reference_number.str.startswith('HK') & (orders.item_description.str.contains('RAIL'))].copy()
vip_orders['y'] = 0
vip_orders['steps'] = 0
vip_mos = vip_orders.index.values.tolist()
vip_logs = logs[(logs.order_number.isin(vip_mos)) & (logs.quantity_complete != 0)].copy()
vip_logs.sort_values('sequence_code', ascending=True, inplace=True)
drop_cols = ['customer_job_number', 'reference_number', 'item_number', 'product_variant', 'product_clearance',  'product_block_count', 'product_g1', 'product_pitch',  'order_status', 'order_release_code', 'order_customer_name', 'order_replied_due', 'order_scheduled_due', 'order_original_due', 'order_scheduled_shipping_date', 'order_class', 'promise_date', 'sales_item_number', 'printed_due']
vip_orders = vip_orders.drop(columns=drop_cols)
# facility_ids = ['B0021', 'F0010', 'B0010', 'K0010', 'K0020', 'B0012', 'B0020', 'B0040', 'B0070', 'B0080', 'C0040', 'C0060', 'C0080', 'C0010','C0020', 'J0020', 'H0020', 'D0010', 'H0070', 'H0010', 'D0020','H0030', 'C0050', 'C0100', 'C0110', 'J0025', 'B0025', 'C0075','B0060', 'B0090', 'C0090', 'B0030', 'C0070', 'K0011', 'H0046','H0050', 'B0050', 'H0060', 'B0015', 'H0045', 'B0062', 'J0060','H0051', 'J0050', 'C0030', 'C0031']
# vip_orders[facility_ids] = 0
vip_orders.head()
vip_logs['time_out'] = pd.to_datetime(vip_logs['time_out'])


In [ ]:
vip_mos = vip_orders.index.values.tolist()
['order_number', 'employee_number', 'quantity_complete',
       'quantity_scrapped', 'facility_id', 'machine_id', 'sequence_code',
       'time_in', 'time_out', 'employee_name', 'employee_first_name',
       'employee_last_name', 'start', 'diff', 'wcount', 'diff2']
vip_orders.head()

In [43]:
import csv
THROTTLE_MIN = 1800
vip_mos = vip_orders.index.values.tolist()
off_days = ['2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-05-30', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06']
o_days = pd.to_datetime(off_days, infer_datetime_format=True, utc=True)
def cal_days (row):
    return len(list(x for x in o_days if (row.start < x < row.time_out)))

def f(mo):       
    mo_logs = vip_logs[vip_logs.order_number == mo].sort_values('sequence_code')    
    mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')    
    mo_logs = mo_logs.drop_duplicates(subset=['facility_id'], keep='first')    
    mo_logs['start'] = mo_logs['time_out'].shift(1)
    mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')                
    mo_logs['wcount'] = 0
    mo_logs['diff2'] = 0
    
    if len(mo_logs[mo_logs['diff'] < 0]):
        print('negative', mo)
        return
    try:                    
        for index, row in mo_logs.iterrows():
            mo_logs[index, 'wcount'] = len(list(x for x in o_days if (row['start'] < x < row['time_out'])))        
        mo_logs['diff2'] = mo_logs['diff'] - mo_logs['wcount'] * 1440        
        mo_logs.loc[mo_logs['diff2'] > THROTTLE_MIN, 'diff2'] = THROTTLE_MIN                
        durations_on_facilities = mo_logs.pivot(index='order_number', columns='facility_id', values='diff2')
        sequence = mo_logs.pivot(index='order_number', columns='facility_id', values='sequence_code')
        return mo, durations_on_facilities, sequence
    except Exception as e: print(mo, e)

def append_res(result):        
    for i in result:
        if i:
            mo, series, sequence = i
            try:                                
                for f in series.columns:
                    vip_orders.loc[mo, f] = series.loc[mo, f]   
                with open('eggs.csv', 'a', newline='') as csvfile:
                    spamwriter = csv.writer(csvfile, delimiter='@')
                    spamwriter.writerow([mo, series.columns.tolist(), series.values, sequence.values[0]])
            except Exception as e: print(e)

if __name__ == "__main__":        
    with mp.Pool(16) as pool:        
        print('pool started')
        results = pool.map_async(f, vip_mos, callback = append_res)
        results.wait() 
print("done")
vip_orders.describe()

pool started
negative M879210
negative M902738
negative M872600
negative M923101
negative M060012
negative M969136
negative M922207
negative M927598
negative M895976
negative M899439
negative M068565
negative M049438
negative M078482
negative M078483
negative M886275
negative M894539
negative M116479
negative M900740
negative M871712
negative M895657
negative M886415
negative M894540
negative M894541
negative M897395
negative M897397
negative M960887

M928337 negativenegative M919830
negative M928009
negative M915134
negative M920891
negative M872738
negative M919761
negative M919764
negative M872739
negative M933331
negative M060025
negative M929168
negative M935036
negative M057556
negative M016467
negative M925883
negative M914975
negative M075409
negative M915698
negative M930818
negative M934760
negative M057588
negative M928840
negative M964317
negative M916584
negative M066972
negative M956163
negative M956803
negative M057425
negative M072303
negative M055843
negative M070177
n

,product_length,order_quantity,y,steps,D0010,D0020,F0010,C0010,C0020,C0060,...,H0060,H0050,B0030,F0020,J0025,B0015,GL030,H0051,0010,B0025
count,39034.000000,39034.000000,39034.0,39034.0,0.0,11950.000000,37923.000000,1.0,22351.000000,16031.000000,...,44.000000,26.0,7.000000,1.0,1.0,66.000000,5.000000,7.0,0.0,10.000000
mean,1186.267492,8.259236,0.0,0.0,NaN,1494.298996,852.696991,1800.0,1090.088810,1191.861456,...,1760.522727,1800.0,1582.285714,1002.0,1800.0,1757.303030,1532.600000,1800.0,NaN,845.400000
std,1384.598633,5.433348,0.0,0.0,NaN,523.861433,691.483327,NaN,555.108159,623.679447,...,261.862603,0.0,576.017857,NaN,NaN,232.454857,597.924577,0.0,NaN,133.799352
min,38.000000,1.000000,0.0,0.0,NaN,0.000000,0.000000,1800.0,0.000000,0.000000,...,63.000000,1800.0,276.000000,1002.0,1800.0,0.000000,463.000000,1800.0,NaN,648.000000
25%,318.000000,4.000000,0.0,0.0,NaN,1412.250000,137.000000,1800.0,626.000000,575.500000,...,1800.000000,1800.0,1800.000000,1002.0,1800.0,1800.000000,1800.000000,1800.0,NaN,768.500000
50%,670.000000,8.000000,0.0,0.0,NaN,1800.000000,830.000000,1800.0,1024.000000,1342.000000,...,1800.000000,1800.0,1800.000000,1002.0,1800.0,1800.000000,1800.000000,1800.0,NaN,841.500000
75%,1600.000000,10.000000,0.0,0.0,NaN,1800.000000,1666.000000,1800.0,1800.000000,1800.000000,...,1800.000000,1800.0,1800.000000,1002.0,1800.0,1800.000000,1800.000000,1800.0,NaN,925.750000
max,56290.000000,50.000000,0.0,0.0,NaN,1800.000000,1800.000000,1800.0,1800.000000,1800.000000,...,1800.000000,1800.0,1800.000000,1002.0,1800.0,1800.000000,1800.000000,1800.0,NaN,1050.000000


In [ ]:

THROTTLE_H = 1880
off_days = ['2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-05-30', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06']
cols_b = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020', 'ZGK', 'ZGP', 'ZLE', 'ZLT', 'order_quantity', 'product_family', 'product_length', 'product_model', 'facility', 'y']
o_days = pd.to_datetime(off_days, infer_datetime_format=True, utc=True)
answer_df = pd.DataFrame(columns=cols_b)

for mo in vip_mos:
    mo_logs = vip_logs[vip_logs.order_number == mo].sort_values('sequence_code')    
    mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')    
    mo_logs = mo_logs.drop_duplicates(subset=['facility_id'], keep='first')    
    mo_logs['start'] = mo_logs['time_out'].shift(1)
    mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')
    mo_logs['wcount'] = 0
    mo_logs['diff2'] = 0
    
    if len(mo_logs[mo_logs['diff'] < 0]):
        print('negative', mo)
        pass
    try:                            
        for r in mo_logs.itertuples():            
            time_out = r[9]
            start = r[13]
            mo_logs[r[0], 'wcount'] = len(list(x for x in o_days if (r[13] < x < r[9])))        

        # for index, row in mo_logs.iterrows():
        #     mo_logs[index, 'wcount'] = len(list(x for x in o_days if (row['start'] < x < row['time_out'])))
        
        mo_logs['diff2'] = mo_logs['diff'] - mo_logs['wcount'] * 1440        
        mo_logs.loc[mo_logs['diff2'] > THROTTLE_H, 'diff2'] = THROTTLE_H                
        durations_on_facilities = mo_logs.pivot(index='order_number', columns='facility_id', values='diff2')                
        durations_on_facilities = mo_logs.pivot(index='order_number', columns='facility_id', values='sequence_code')
        for f in durations_on_facilities.columns:
            vip_orders.loc[mo, f] = durations_on_facilities.loc[mo, f]
        
    except Exception as e: print(mo, e)

In [ ]:
vip_orders['y'] = 0
vip_orders['facility'] = 0
vip_orders.head(50)

In [ ]:
b_types = ['-III', '(GK)', '(GP)', '-II']
p_types = ['K','E','P','F','T','S','M','H']
vip_orders = vip_orders[vip_orders['y'] > 3]
des = vip_orders['item_description']

In [ ]:
test = vip_orders.copy()
test['ZLE'] = np.where(test['item_description'].str.count('LE'), 1, 0)
test['ZGK'] = np.where(test['item_description'].str.count('GK'), 1, 0)
test['ZLT'] = np.where(test['item_description'].str.count('LT'), 1, 0)
test['ZGP'] = np.where(test['item_description'].str.count('GP'), 1, 0)
# cols_a = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0062', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020', 'ZGK', 'ZGP', 'ZLE', 'ZLT', ]
cols_b = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020', 'ZGK', 'ZGP', 'ZLE', 'ZLT', 'order_quantity', 'product_family', 'product_length', 'product_model', 'facility', 'y']
# cols_b = ['ZGK', 'ZGP', 'ZLE', 'ZLT', 'order_quantity', 'product_family', 'product_length', 'product_model', 'steps', 'y']
# cols_c = ['ZGK', 'ZGP', 'ZLE', 'ZLT', 'order_quantity', 'product_family', 'product_length', 'product_model', 'steps', 'y']
test = test[cols_b]
test['product_length'] = test['product_length'] / 1000
test.fillna(0, inplace=True)
print(len(test.columns))
dimension = 48

In [ ]:
cols_a = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020']
len(cols_a)

In [ ]:
test1 = test.copy()
list_classify = ["product_family", "product_model"]
cols_d = ['ZGK', 'ZGP', 'ZLE', 'ZLT', 'order_quantity', 'product_family', 'product_length', 'product_model', 'facility', 'y']
dataset = pd.DataFrame(columns=cols_d)
for i in list_classify:    
    test1[i], uniques = pd.factorize(test1[i], sort=True)    
    print(uniques)
for r in zip(*test1.to_dict("list").values()):
    arr = np.array(r[:48], int)
    x = np.array(r[48:])
    for i in np.where(arr > 0.)[0]:        
        x[-2] = i
        x[-1] = arr[i]        
        dataset = pd.concat([dataset, pd.DataFrame([x], columns=cols_d)], ignore_index=True)
        # dataset.append(pd.DataFrame([x], columns=cols_d), ignore_index=True)  

In [ ]:
dataset.to_csv("single_stage.csv")

In [ ]:
arr = np.array((0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1256.0, 1088.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 20.0, 4, 0.4, 4, 0), int)
arr = arr[:48]
np.where(arr > 0.)

In [ ]:

shuffled = dataset.sample(frac=1)
pivot = int(len(dataset) * .2)
train = shuffled.iloc[pivot:].copy()
val = shuffled.iloc[:pivot].copy()
train = train.astype(float)
val = val.astype(float)

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

class MyDataset(Dataset):   
  def __init__(self,data):     
    super(MyDataset).__init__()
    x=data.iloc[:,0:9].values
    y=data.iloc[:,9].values     
    self.x_train=torch.tensor(x,dtype=torch.float32).cuda()    
    self.y_train=torch.tensor(y,dtype=torch.float32).view(-1, 1).cuda()    
 
  def __len__(self):
    return len(self.y_train)   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]
 
  def __len__(self):
    return len(self.x_train)   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

train_set=MyDataset(train)
val_set=MyDataset(val)
train_dl=DataLoader(train_set, batch_size=256, shuffle=True, num_workers=0)
val_dl=DataLoader(val_set, batch_size=len(val_set))

In [ ]:
loss_fn = F.mse_loss
def fit(num_epochs, model, loss_fn, opt):
    val_best  = 26
    for epoch in range(num_epochs):        
        for xb,yb in train_dl:            
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if epoch % 10 == 0:
            loss_val = loss.item()
            print(loss_val, epoch)            
            for v_xb,v_yb in val_dl:
                result = model(v_xb)     
                val_err = torch.mean(abs(v_yb - result) / v_yb).item() * 100                    
                print("val_err: ", val_err)
            if val_err < val_best:
                val_best = val_err
                torch.save(model.state_dict(), f"throttle_{str(val_err).replace('.', '_')[:5]}.pth")
    
class SimpleNet(nn.Module):    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(9, 128)                
        self.linear2 = nn.Linear(128, 32)        
        self.linear3 = nn.Linear(32, 1)
        
        self.act1 = nn.ReLU() # Activation function        
        self.act2 = nn.ReLU() # Activation function        
        
            
    # Perform the computation
    def forward(self, x):                
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        x = self.linear3(x)
        return x

model = SimpleNet().cuda()
opt = torch.optim.SGD(model.parameters(), 1e-6, momentum=0.9)
fit(1000, model, loss_fn, opt)

In [ ]:
len(test)

In [ ]:
# low_days = ['2021-07-03', '2021-07-04', '2021-07-10', '2021-07-11', '2021-07-17', '2021-07-18', '2021-07-24', '2021-07-25', '2021-07-31', '2021-08-01', '2021-08-07', '2021-08-08', '2021-08-14', '2021-08-15', '2021-08-21', '2021-08-22', '2021-08-28', '2021-08-29', '2021-09-04', '2021-09-05', '2021-09-11', '2021-09-12', '2021-09-18', '2021-09-19', '2021-09-25', '2021-09-26', '2021-10-02', '2021-10-03', '2021-10-09', '2021-10-10', '2021-10-16', '2021-10-17', '2021-10-23', '2021-10-24', '2021-10-30', '2021-10-31', '2021-11-06', '2021-11-07', '2021-11-13', '2021-11-14', '2021-11-20', '2021-11-21', '2021-11-27', '2021-11-28', '2021-12-04', '2021-12-05', '2021-12-11', '2021-12-12', '2021-12-18', '2021-12-19', '2021-12-25', '2021-12-26', '2021/06/05', '2021/06/06', '2021/06/13', '2021/06/19', '2021/06/20', '2021/06/27', '2021/06/30', '2021/07/02', '2021/07/03', '2021/07/04', '2021/07/05', '2021/07/11', '2021/07/17', '2021/07/18', '2021/07/25', '2021/07/31', '2021/08/01', '2021/08/08', '2021/08/14', '2021/08/15', '2021/08/22', '2021/08/28', '2021/08/29', '2021/09/04', '2021/09/05', '2021/09/06', '2021/09/12', '2021/09/18', '2021/09/19', '2021/09/26', '2021/10/02', '2021/10/03', '2021/10/10', '2021/10/16', '2021/10/17', '2021/10/24', '2021/10/30', '2021/10/31', '2021/11/07', '2021/11/13', '2021/11/14', '2021/11/21', '2021/11/25', '2021/11/26', '2021/11/27', '2021/11/28', '2021/12/05', '2021/12/11', '2021/12/12', '2021/12/19', '2021/12/23', '2021/12/24', '2021/12/25', '2021/12/26', '2021/12/31', '2022-01-01', '2022-01-02', '2022-01-08', '2022-01-09', '2022-01-15', '2022-01-16', '2022-01-22', '2022-01-23', '2022-01-29', '2022-01-30', '2022-02-05', '2022-02-06', '2022-02-12', '2022-02-13', '2022-02-19', '2022-02-20', '2022-02-26', '2022-02-27', '2022-03-05', '2022-03-06', '2022-03-12', '2022-03-13', '2022-03-19', '2022-03-20', '2022-03-26', '2022-03-27', '2022-04-02', '2022-04-03', '2022-04-09', '2022-04-10', '2022-04-16', '2022-04-17', '2022-04-23', '2022-04-24', '2022-04-30', '2022-05-01', '2022-05-07', '2022-05-08', '2022-05-14', '2022-05-15', '2022-05-21', '2022-05-22', '2022-05-28', '2022-05-29', '2022-06-04', '2022-06-05', '2022-06-11', '2022-06-12', '2022-06-18', '2022-06-19', '2022-06-25', '2022-06-26', '2022-07-02', '2022-07-03', '2022-07-09', '2022-07-10', '2022-07-16', '2022-07-17', '2022-07-23', '2022-07-24', '2022-07-30', '2022-07-31', '2022-08-06', '2022-08-07', '2022-08-13', '2022-08-14', '2022-08-20', '2022-08-21', '2022-08-27', '2022-08-28', '2022-09-03', '2022-09-04', '2022-09-10', '2022-09-11', '2022-09-17', '2022-09-18', '2022-09-24', '2022-09-25', '2022-10-01', '2022-10-02', '2022-10-08', '2022-10-09', '2022-10-15', '2022-10-16', '2022-10-22', '2022-10-23', '2022-10-29', '2022-10-30', '2022-11-05', '2022-11-06', '2022/01/01', '2022/01/02', '2022/01/03', '2022/01/08', '2022/01/09', '2022/01/16', '2022/01/22', '2022/01/23', '2022/01/30', '2022/02/04', '2022/02/05', '2022/02/06', '2022/02/13', '2022/02/19', '2022/02/20', '2022/02/27', '2022/03/05', '2022/03/06', '2022/03/12', '2022/03/13', '2022/03/19', '2022/03/20', '2022/03/27', '2022/04/02', '2022/04/03', '2022/04/10', '2022/04/15', '2022/04/16', '2022/04/17', '2022/04/23', '2022/04/24', '2022/04/30', '2022/05/01', '2022/05/08', '2022/05/14', '2022/05/15', '2022/05/22', '2022/05/28', '2022/05/29', '2022/05/30', '2022/06/05', '2022/06/11', '2022/06/12', '2022/06/18', '2022/06/19', '2022/06/25', '2022/06/26', '2022/06/30', '2022/07/02', '2022/07/03', '2022/07/04', '2022/07/05', '2022/07/10', '2022/07/16', '2022/07/17', '2022/07/23', '2022/07/24', '2022/07/30', '2022/07/31', '2022/08/06', '2022/08/07', '2022/08/13', '2022/08/14', '2022/08/20', '2022/08/21', '2022/08/27', '2022/08/28', '2022/09/03', '2022/09/04', '2022/09/05', '2022/09/10', '2022/09/11', '2022/09/17', '2022/09/18', '2022/09/24', '2022/09/25', '2022/10/01', '2022/10/02', '2022/10/08', '2022/10/09', '2022/10/15', '2022/10/16', '2022/10/22', '2022/10/23', '2022/10/29', '2022/10/30', '2022/11/05', '2022/11/06']
# def f(mo):       
#     mo_logs = vip_logs[vip_logs.order_number == mo].copy()
#     mo_logs = mo_logs.drop_duplicates(subset=['sequence_code'], keep='first')
#     mo_logs['diff'] = mo_logs['time_out'].diff(periods=1).astype('timedelta64[m]')
#     mo_logs['start'] = mo_logs['time_out'].shift(1)    
#     mo_logs.loc[mo_logs['diff'] > THROTTLE_MIN, 'diff'] = THROTTLE_MIN
#     try:
#         # for index, row in mo_logs.iloc[1:].iterrows():                
#         #     if row['diff'] > THROTTLE_MIN:
#         #         end = row.time_out.strftime("%Y-%m-%d")
#         #         start = row.start.strftime("%Y-%m-%d")
#         #         offday_count = len(list(x for x in low_days if start < x < end))             
#         #         row['diff'] -= offday_count * 1440
#         #         if row['diff'] > THROTTLE_MIN:
#         #             # print(index)
#         #             mo_logs.loc[index, 'diff'] = THROTTLE_MIN    
#         period = mo_logs['diff'].sum() / 60
#         # return mo, period, mo_logs.iloc[0]['time_out']
#         return mo, period, len(mo_logs)
#     except Exception as e: print(mo, e)

# b_types = ['-III', '(GK)', '(GP)', '-II']
# p_types = ['K','E','P','F','T','S','M','H']
# for i in b_types:
#     vip_orders[i] = 0
# for i in p_types:
#     vip_orders[i] = 0
# vip_orders['custom'] = 0
# for index, row in vip_orders.iterrows():
#     try:
#         des = row['item_description'].split(' RAIL')[0].split('L')[1]
#         for bt in b_types:
#             if bt in des:
#                 vip_orders.loc[index, bt] = 1
#                 des = des.replace(bt, '')
#         print(des)
#         for pt in p_types:
#             if pt in des:            
#                 vip_orders.loc[index, pt] = 1
#     except Exception as e:
#         vip_orders.loc[index, 'custom'] = 1